# TRABAJO PRÁCTICO 9 - NAIVE BAYES

In [928]:
#librerias

#Generales

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import logsumexp

#Descarga del catálogo
import urllib.request
import tarfile

#Manejo de los epubs
import os
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
from collections import Counter


import nltk
from nltk.corpus import stopwords


# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score

# Procesamiento del catálogo

#### Descarga del catálogo

In [929]:
# Se colocan las URLs de los archivos a descargar
url_catalog = "https://web.csc.gob.ar/~jzuloaga/epub/catalog.csv"
url_tar = "https://web.csc.gob.ar/~jzuloaga/epub/compressed.tar"

# Se colocan nombres a los archivos
file_catalog = "catalog.csv"
file_tar = "compressed.tar"

# Se descargan solo si no existen
if not os.path.exists(file_catalog):
    print("Descargando catalog.csv...")
    urllib.request.urlretrieve(url_catalog, file_catalog)

if not os.path.exists(file_tar):
    print("Descargando compressed.tar...")
    urllib.request.urlretrieve(url_tar, file_tar)

print("Descarga de archivos completa.")

# Descompresion del catalogo
# Se descomprimen los archivos
if not os.path.exists("compressed"):
    with tarfile.open(file_tar, "r") as tar:
        tar.extractall("compressed")
        print("Archivos extraídos en la carpeta 'compressed'\n")
else:
    print("La carpeta 'compressed' ya existe, no se vuelve a extraer.\n")

Descargando compressed.tar...
Descarga de archivos completa.
La carpeta 'compressed' ya existe, no se vuelve a extraer.



#### Cargar el catálogo y explorar el contenido de sus columnas.  ¿Qué representa cada una?

In [930]:
# Se carga el catalogo
df_catalog = pd.read_csv("catalog.csv")

# Se muestran las columnas del catalogo
print("Columnas del catálogo:")
print(df_catalog.columns)


Columnas del catálogo:
Index(['EPL Id', 'Título', 'Autor', 'Géneros', 'Colección', 'Volumen',
       'Año publicación', 'Sinopsis', 'Páginas', 'Revisión', 'Idioma',
       'Publicado', 'Estado', 'Valoración', 'Nº Votos', 'Enlace(s)'],
      dtype='object')


#### Significado de las columnas

EPL Id: Id del libro en el catálogo.

Título: Título del libro.

Autor: Autor del libro.

Géneros: Géneros del libro (una lista de géneros para cada libro).

Colección: Colección, serie o saga a la que pertenece el libro.

Volumen: Número del volumen del libro dentro de la colección.

Año publicación: Año de publicación del libro.

Sinopsis: Descripción o resumen del contenido del libro.

Páginas: Cantidad de páginas.

Revisión: Información editorial o revisión del texto.

Idioma: Idioma del libro.

Publicado: Estado o fecha de publicación.

Estado: Estado del libro dentro del catálogo.

Valoración: Valoración promedio de los usuarios.

Nº Votos: Cantidad de votos recibidos por los usuarios.

Enlace(s): Enlaces relacionados.

#### Filtrar las entradas del catálogo, de manera de quedarse solamente con los libros en idioma español.

In [931]:
df_catalog = df_catalog[df_catalog['Idioma'] == 'Español']

#### Limitar las entradas del catálogo a las que tenga su correspondiente libro digital.

In [932]:
path_books = 'compressed/compressed'

# Se crea una lista con los epubs en path_books
epubs = [f for f in os.listdir(path_books) if f.endswith(".epub")]

print(f"Total de archivos EPUB disponibles: {len(epubs)}")

# Se crea un conjunto de ids válidos, es decir, se acumulan los ids de los epubs que existen en la ruta path_books
valids_id = set()
for f in epubs:
    try:
        valids_id.add(int(f.replace(".epub", "")))
    except ValueError:
        pass

# Se reduce el catálogo a aquellos libros que tienen su versión digital 
df_catalog = df_catalog[df_catalog['EPL Id'].isin(valids_id)].copy()


Total de archivos EPUB disponibles: 8958


# Definición de las clases

#### Analizar la distribución de libros por categoría.

In [933]:
# Se separan los géneros y se cuentan individualmente
all_genres = df_catalog['Géneros'].dropna().str.split(r',\s*')

# Se cuentan los géneros, utilizando una doble compresión.
# se toma cada sublista (es decir, cada lista de géneros para cada libro) en all_genres y a cada una de ellas, se les toma cada uno de los géneros
# y se aplanan en una lista, donde Counter cuenta la cantidad de ocurrencias de cada uno de los géneros en la lista total 
genre_counter = Counter([g for sublist in all_genres for g in sublist])

# Se crea un Series de pandas y se ordenan las ocurrencias de cada género, de manera descendente
genre_proportion = pd.Series(genre_counter).sort_values(ascending=False)  / len(all_genres)

genre_proportion.head(10)

Drama         0.134070
Otros         0.118218
Aventuras     0.111632
Policial      0.111409
Realista      0.095334
Intriga       0.076914
Histórico     0.065975
Filosofía     0.061733
Historia      0.059500
Fantástico    0.052467
dtype: float64

#### Eliminar el género Otros por ser una categoría redundante.

In [934]:
# Se eliminan los libros con únicamente el género "Otros"
df_catalog = df_catalog[~df_catalog['Géneros'].str.fullmatch(r'Otros', na=False)]

# Se elimina el género "Otros" de los demás libros
df_catalog['Géneros'] = (
    df_catalog['Géneros']
    .str.replace(r',?\s*Otros', '', regex=True)
    .str.strip(', ')
)

#### Proponer y justificar un criterio para elegir un único género cuando un libro tenga varios.

In [935]:
# Se define una función para seleccionar el género con menor aparición global para cada libro
def pick_rarest_genre(genres):
    if pd.isna(genres):
        return None
    genre_list = [g.strip() for g in genres.split(',')]
    # Ordenar por frecuencia (de menor a mayor)
    rarest = min(genre_list, key=lambda g: genre_counter.get(g, 0))
    return rarest

# Se crea una columna con el género seleccionado para cada libro
df_catalog['Género_único'] = df_catalog['Géneros'].apply(pick_rarest_genre)

# Se elimina la columna "Géneros" del catálogo
df_catalog.drop(columns=['Géneros'], inplace=True)

#### Reportar la distribución final de libros por categoría

In [936]:
# Se cuenta la cantidad de apariciones de cada género en la columna de 'Genero_unico'
final_distribution = df_catalog['Género_único'].value_counts()

final_distribution = final_distribution / df_catalog.shape[0]

final_distribution.head(10)

Género_único
Realista           0.085018
Drama              0.081098
Policial           0.076320
Aventuras          0.074237
Intriga            0.073012
Histórico          0.058434
Historia           0.047287
Filosofía          0.044469
Fantástico         0.034669
Ciencia ficción    0.034179
Name: count, dtype: float64

#### Separar los libros para definir conjuntos de entrenamiento y testeo utilizando las proporciones 75/25. Fijar la semilla para reproducibilidad utilizando su número de padrón.

In [937]:
# Se extraen los géneros únicos
unique_genre = df_catalog['Género_único'].unique()

# Se genera un diccionario con una etiqueta para cada genero (1 : unique_genre +1)
dict_genre = {genero: idx + 1 for idx, genero in enumerate(unique_genre)}

# Se cargan las etiquetas para cada género en el df
df_catalog['Etiqueta_género'] = df_catalog['Género_único'].map(dict_genre)

# Los datos de entrada, son seleccionados sin incluir las etiquetas ni su género
X = df_catalog.drop(columns=['Etiqueta_género', 'Género_único']) 

# Datos a predecir
y = df_catalog["Etiqueta_género"]

# Número de padrón
student_id = 104241

# Se separan en datos de entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=student_id,
)

print('Tamaño del catálogo completo:', df_catalog.shape)
print('Tamaño del conjunto de entrenamiento:', X_train.shape)
print('Tamaño del conjunto de testeo:', X_test.shape)

Tamaño del catálogo completo: (8163, 17)
Tamaño del conjunto de entrenamiento: (6122, 15)
Tamaño del conjunto de testeo: (2041, 15)


# Preprocesamiento de texto

#### Funciones

In [938]:
path_books = 'compressed/compressed'

def get_text_epub(epub_path):

    try:
        # Se abre el archivo epub
        book = epub.read_epub(epub_path)
        
        chapters_list = []

        # Se recorren todos los documentos XHTML del libro
        for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
            
            # Se obtiene el contenido del documento XHTML
            xhtml_content = item.get_content()
            
            # Se usa BeautifulSoup para limpiar el xhtml y solo obtener el texto
            soup = BeautifulSoup(xhtml_content, 'html.parser')
            
            # Se limpia el texto extraído
            clean_text = soup.get_text()
            
            chapters_list.append(clean_text)
        
        # Se unifican los capitulos en un string
        return " ".join(chapters_list)

    except FileNotFoundError:
        print(f"No se encontró el archivo: {epub_path}")
        return None
    
    except Exception as e:
        print(f"Error procesando (corrupto): {epub_path}: {e}")
        return None
    


def get_text_epub_id (id_book):
    
    # Se construye la ruta completa al libro epub
    filename = f"{id_book}.epub"
    
    epub_path = os.path.join(path_books, filename)

    return get_text_epub(epub_path)

#### El formato de libros epub es un archivo comprimido zip que contiene la metadata y estructura del libro, archivos multimedia y archivos xhtml con el texto del libro. Extraer el texto de esos archivos. Podrá realizarlo manualmente o valerse de bibliotecas.

In [939]:
n_samples = 100

# Se toman unas pocas muestras para evitar ejecutar con todos los archivos juntos. Se toman por índices para conservar alineados los datos.
X_train = X_train.iloc[:n_samples].copy()
y_train = y_train.iloc[:n_samples].copy()
X_test = X_test.iloc[:n_samples].copy()
y_test = y_test.iloc[:n_samples].copy()

print(f"Libros en X_train: {len(X_train)}")
print(f"Libros en X_test: {len(X_test)}")


# Se cargan en X_train y X_test los textos de cada uno de los libros, utilizando su ID
X_train.loc[:, 'texto'] = X_train['EPL Id'].apply(get_text_epub_id)
X_test.loc[:, 'texto'] = X_test['EPL Id'].apply(get_text_epub_id)

print("Se completó la extracción de los textos")

# Se eliminan las filas que no tienen texto (vacías o con NaN)
X_train_clean = X_train.dropna(subset=['texto'])
X_test_clean = X_test.dropna(subset=['texto'])

# Se seleccionan las etiquetas que corresponden a libros cuyo texto no está vacío
y_train_clean = y_train.loc[X_train_clean.index]
y_test_clean = y_test.loc[X_test_clean.index]

print('Datos válidos para X_train:', X_train_clean.shape)
print('Datos válidos para X_test:', X_test_clean.shape)

print('Datos válidos para y_train:', y_train_clean.shape)
print('Datos válidos para y_test:', y_test_clean.shape)


Libros en X_train: 100
Libros en X_test: 100
Error procesando (corrupto): compressed/compressed\58986.epub: "There is no item named 'OEBPS/Text/autor.xhtml' in the archive"
Error procesando (corrupto): compressed/compressed\40832.epub: "There is no item named 'META-INF/container.xml' in the archive"
Se completó la extracción de los textos
Datos válidos para X_train: (98, 16)
Datos válidos para X_test: (100, 16)
Datos válidos para y_train: (98,)
Datos válidos para y_test: (100,)


#### Aplicar CountVectorizer (sklearn) al texto. Ajustar max_df, min_df y stop_words a criterio personal, justificando las decisiones. El corpus de texto completo es demasiado extenso para la memoria. Se sugiere el uso de Generators para procesar el texto plano on-demand

In [940]:
# Se declara el generator, para procesar el texto por partes
def text_generator(df_clean):
    for texto in df_clean['texto']:
        yield texto

# Se descargan las stopwords
nltk.download('stopwords')

# Se definen las stopwords en español
spanish_stopwords = stopwords.words('spanish')

# Se crea el vectorizador utilizando las stopwords en español (Palabras que se ignoran del texto completo).
# Se cuentan las apariciones de las palabras en los textos, 
# siendo guardados en una matriz de shape: (n_libros, n_palabras_del_vocabulario) cada posición, almacena esa cantidad de apariciones
# Se seleccionan los porcentajes para min_df y max_df ya que se considera que palabras demasiado
# poco frecuentes o demasiado frecuentes en el total de los libros, no aportan información significativa para distinguir entre
# un género y otro.
vectorizer = CountVectorizer(
    stop_words=spanish_stopwords,
    max_df=0.9,    # Se descartan las palabras más frecuentes, las que aparecen en más del 90% de los libros
    min_df= 0.01   # Se descartan las palabras que aparecen en menos del 1% de los libros
)


# Se entrena el vectorizador con todos los textos de los libros
train_gen = list(text_generator(X_train_clean))
X_train_vec = vectorizer.fit_transform(train_gen)

test_gen = list(text_generator(X_test_clean))
X_test_vec = vectorizer.transform(test_gen)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\solek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Se desea comprobar el funcionamiento del vectorizador. Para ello, aplicar el vectorizador ya entrenado a dos obras clásicas del catálogo de diferentes géneros (por ejemplo, “Estudio en escarlata” y “Orgullo y prejuicio”). Descartar las palabras presentes en ambos libros. Luego, para cada libro, reportar las 40 palabras más frecuentes. Interpretar los resultados obtenidos.

In [941]:
# Se extraen los textos de 2 libros diferentes
text_alice = X_train_clean.loc[X_train_clean['Título'] == 'Alicia en el país de las maravillas (il. de Marta Gómez-Pintado)', 'texto'].iloc[0]
text_spain   = X_train_clean.loc[X_train_clean['Título'] == 'Historia de los heterodoxos españoles', 'texto'].iloc[0]


# Se vectorizan los textos de cada libro (shapes: 1xn_vocabulario)
X_alice = vectorizer.transform([text_alice])
X_spain   = vectorizer.transform([text_spain])


freq_alice = np.squeeze(np.asarray(X_alice.toarray()))
freq_spain   = np.squeeze(np.asarray(X_spain.toarray()))

# Se obtienen los índices donde realmente hay palabras
idx_alice = set(np.where(freq_alice > 0)[0])
idx_spain   = set(np.where(freq_spain > 0)[0])

# Se obtienen las palabras en común
common_words = idx_alice.intersection(idx_spain)

# Se crean listas de las palabras únicas en cada libro
words_alice_only = list(idx_alice - common_words)
words_spain_only   = list(idx_spain - common_words)

# Se obtiene el vocabulario en el vectorizador
vocabulary = vectorizer.get_feature_names_out()

# Se obtiene la cantidad de apariciones de cada palabra en cada libro
freq_alice_filtered = [(vocabulary[i], freq_alice[i]) for i in words_alice_only]
top_alice = sorted(freq_alice_filtered, key=lambda x: x[1], reverse=True)[:40]

freq_spain_filtered = [(vocabulary[i], freq_spain[i]) for i in words_spain_only]
top_spain = sorted(freq_spain_filtered, key=lambda x: x[1], reverse=True)[:40]


print("Palabras más frecuentes en 'Alicia en el país de las maravillas' (excluyendo las que se comparten):")
for word, freq in top_alice:
        print(word, '->', freq)

print("\nPalabras más frecuentes en 'Historia de los heterodoxos españoles' (excluyendo las que se comparten):")
for word, freq in top_spain:
        print(word, '->', freq)


shared = set(top_alice).intersection(top_spain)
print('Entre los tops de palabras más frecuentes, se comparten:', shared)

# print(X_alice.shape)
# print(X_spain.shape)
# print(vocabulary.shape)

Palabras más frecuentes en 'Alicia en el país de las maravillas' (excluyendo las que se comparten):
alicia -> 433
tortuga -> 64
grifo -> 55
conejo -> 53
lirón -> 39
gustaría -> 20
bebé -> 14
cocinera -> 13
corten -> 13
dodo -> 11
mantequilla -> 11
moraleja -> 10
enseguida -> 10
croquet -> 10
jardineros -> 10
cheshire -> 9
pizarras -> 8
melaza -> 8
pescadilla -> 8
sonrisa -> 8
cogió -> 8
pizca -> 7
nadar -> 7
lagartija -> 7
temblorosa -> 7
ansiedad -> 7
centímetros -> 7
carroll -> 6
cerdito -> 6
señorita -> 6
chilló -> 6
enfadado -> 6
barbilla -> 6
debería -> 6
medía -> 5
ujieres -> 5
gata -> 5
cambiado -> 5
dedal -> 5
gruñido -> 5

Palabras más frecuentes en 'Historia de los heterodoxos españoles' (excluyendo las que se comparten):
et -> 2822
san -> 1633
españa -> 1510
juan -> 1226
iglesia -> 1197
siglo -> 1033
non -> 846
fe -> 801
fr -> 786
doctrina -> 740
obispo -> 720
espíritu -> 689
santo -> 685
pedro -> 679
etc -> 645
ad -> 625
madrid -> 620
inquisición -> 527
filosofía -> 521
cf 

Dado que se eligieron 2 libros de temáticas muy diferentes, es esperable que ninguno de los libros compartan palabras en su top de 40 palabras más usadas.

# Multinomial Naive Bayes

#### Sobre Naive Bayes (como clasificador): 

Es un clasificador probabilístico, que busca estimar la probabilidad de que un dato $x$ pertenezca a una clase $y$ y clasifica según qué clase tiene mayor probabilidad.

En este caso, los datos son el texto de los libros, mientras que las clases, son los géneros de cada libro. Entonces, se busca que, dado un texto de un libro, se pueda predecir a qué género pertenece.

#### Sobre la hipótesis Naive:

La hipótesis Naive se basa en asumir que todas las palabras son independientes entre sí, dado que ya conocemos la clase de cada una.
Si, por ejemplo, tenemos el libro "Alicia en el país de las maravillas", la aparición de "conejo" y "alicia" es independiente entre sí dado el género.
Lo cual, no es estrictamente cierto.

#### Sobre Naive Bayes Multinomial:

En este modelo, la probabilidad asignada a cada clase (género del libro), dado que se tiene la muestra x (el texto del libro en este caso), es:

$$
p(y \mid x) \propto p(y) \prod_{i=1}^{n} p(x_i \mid y)
$$

Esto nace del teorema de Bayes:

$$
P(y \mid x) = \frac{P(x \mid y) \, P(y)}{P(x)}
$$

Donde:

$y$ es la clase, en este caso el género del libro.

x es el texto de un libro.

$P(y \mid x)$ Es la probabilidad de que el libro sea de la clase $y$ dado el texto del libro (lo que se busca predecir).

$P(y)$ Es la probabilidad a priori de la clase $y$ (en este caso, la proporción de libros de cada género).

$P(x)$ Es la probabilidad marginal de observar esas palabras.

$P(x \mid y)$ Es la probabilidad de observar las palabras x, dado que el libro es del género $y$.

La hipótesis Naive, entra a la hora de calcular $P(x \mid y)$, ya que, dado que las palabras $x_i$ se consideran independientes dado el género al que pertenece el texto del libro, entonces:

$$
P(x \mid y) = \prod_{i=1}^{n} P(x_i \mid y)
$$


Para el caso de la multinomial, esta probabilidad (sin considerar la marginal de x) puede escribirse como:

$p( y = k | x ) \propto c_y \cdot \prod_{j=1}^{V}(\theta_{j}^{(k)})^{N_j}$ 

Donde:

$y$ es la clase, en este caso el género del libro.

x es el texto de un libro, que contiene d palabras: x=($x_1$, $x_2$, ... , $x_d$).

$c_y$ es la probabilidad a priori de cada clase (es decir, la cantidad de líbros de un género con respecto al total de libros).

$\theta_j$ son las probabilidades de que se encuentre en el texto la palabra $j$ ($j$ va desde 1 hasta $V$, es decir, abarca todo el vocabulario), dado el género del libro es $y$

$N_j$ es la cantidad de veces que aparece la palabra j en el libro.


Luego, dado que las probabilidades pueden volverse muy pequeñas y causar inestabilidad numérica, se trabaja con el logaritmo de la probabilidad:

$log (p( y = k | x )) = cte + log(c_y) +\sum_{j=1}^{V}(N_j \cdot log(\theta_{j}^{k})) $



Entrenamiento del modelo:

Para el entrenamiento del modelo, se comienza calculando las probabilidades a priori de cada clase, es decir, la cantidad de libros cada género con respecto al total de libros:

$$
c_k = p(y=k) = \frac{\#\{y_i=k\}}{n}
$$

donde:

$k$ es la clase o género

$i$ es el i-ésimo libro 

$y_i$ es el género del i-ésimo libro

$n$ es el total de libros

luego, se calcula el logaritmo de esta probabilidad, para preparar el cálculo de predict_proba

el paso siguiente, consiste en calcular $\theta$ :

$$
\theta^{k}_{j}
=
\frac{N_{kj} + \alpha_{j}}
{\sum_{m=1}^{V} (N_{km} + \alpha_{m})}
$$



#### Utilizando solamente numpy y scipy, implementar el clasificador MNB. El mismo debe contener los métodos fit, predict y predict_proba.

In [942]:
class MNB:

    def __init__(self, alpha):
        self.classes = None
        self.class_count = None
        self.class_log_prior = None
        self.words_per_genre = None
        self.log_theta = None
        self.alpha = alpha

    # Entrenamiento
    def fit(self, X, y):

        # Se convierte la entrada en un array de numpy
        y = np.asarray(y)

        # Se guardan las clases utilizando los identificadores de los géneros
        self.classes = np.unique(y)

        # Se guarda la cantidad de clases existentes (cantidad de géneros)
        K = len(self.classes)

        # Se carga N como al cantidad de libros que están vectorizados y V como la cantidad de palabras del vocabulario
        N = X.shape[0]
        V = X.shape[1]

        # Se calculan cuantos libros pertenecen a cada género
        self.class_count = np.array([(y == c).sum() for c in self.classes], dtype=float)

        # Se calcula el logaritmo de la probabilidad a priori: log (# libros por género / # libros totales) (para evitar inestabilidad numérica)
        self.class_log_prior = np.log(self.class_count / N)

        # Se crea un vector de ceros de K x V, para contar cuantas veces aparece cada palabra dentro de cada clase
        # Es decir, se cuenta cuantas veces aparece una palabra en un libro de un determinado género
        self.words_per_genre = np.zeros((K, V), dtype=float)

        # Se realiza la cuenta de las palabras presentes en cada género para todos los géneros    
        for idx, c in enumerate(self.classes):
            self.words_per_genre[idx] = X[y == c].sum(axis=0)

        # Se genera el vector de alphas: si es un escalar se llena el array de V alphas con el valor dado
        # en caso de que sea una lista (o similares), se lo convierte a array
        if np.isscalar(self.alpha):
            alpha_vec = np.full(V, self.alpha)
        else:
            alpha_vec = np.asarray(self.alpha)

        # Se calcula:  N_kj + alpha_j
        numerator = self.words_per_genre + alpha_vec  # shape (K, V)

        # Se calcula: sum(N_km + alpha_m) (m va entre 1 y V)
        denominator = numerator.sum(axis=1, keepdims=True)  # (K, 1)

        # Se calculan los thetas (probabilidades de que aparezcan  cada una de las palabras del vocabulario dada cada una de las clases)
        theta_hat = numerator / denominator

        # Se calculan las log-probabilidades para los thetas
        self.log_theta = np.log(theta_hat)


        print('log_theta', self.log_theta.shape)

        return self

    # Predicción soft
    def predict_proba(self, X):

        # Dado que cada posición de la vectorización del textos es: X[i, m] = N_m
        # Entonces, se usa el producto matricial @ para el cálculo
        # Además, se suma el logaritmo de la probabilidad a priori
        # todo esto para calcular el logaritmo de la probabilidad de un libro pertenezca a un determinado género, dado el texto del mismo
        log_prob = X @ self.log_theta.T + self.class_log_prior

        # Se normaliza log_prob, evitando inestabilidad numérica
        log_norm = logsumexp(log_prob, axis=1, keepdims=True)

        # Se vuelve al espacio de probabilidades finalmente
        return np.exp(log_prob - log_norm)

    # Predicción hard
    def predict(self, X):
        return self.classes[np.argmax(self.predict_proba(X), axis=1)]

 #### Reportar el Accuracy y el Macro F1, tanto para entrenamiento como testeo. ¿Cuál sería la probabilidad de error asociada a un clasificador dummy en esta tarea?

In [943]:
model = MNB(alpha = 0.1)

model.fit(X_train_vec,y_train_clean)

# Se obtienen las predicciones del modelo
y_pred_train = model.predict(X_train_vec)
y_pred_test  = model.predict(X_test_vec)

# Se obtienen los accuracy
acc_train = accuracy_score(y_train_clean, y_pred_train)
acc_test  = accuracy_score(y_test_clean, y_pred_test)

# Se obtienen los Macro F1
f1_train = f1_score(y_train_clean, y_pred_train, average='macro')
f1_test  = f1_score(y_test_clean, y_pred_test, average='macro')

print("Métricas")
print(f'Accuracy (train): {round(acc_train *100,3)}%' )
print(f'Accuracy (test): {round(acc_test *100,3)}%')
print(f'Macro F1 (train): {round(f1_train*100,3)}%')
print(f'Macro F1 (test): {round(f1_test*100,3)}%')

log_theta (30, 196426)
Métricas
Accuracy (train): 100.0%
Accuracy (test): 28.0%
Macro F1 (train): 100.0%
Macro F1 (test): 13.407%


#### Se desea efectuar un análisis cualitativo de los errores de clasificación. Para ello, seleccione las 10 obras más populares (de testeo) que hayan sido clasificadas incorrectamente. Interpretar los resultados.

In [944]:
# Se genera un dataset de testeo para trabajar cómodamente
df_test_clean = X_test_clean.copy()
df_test_clean["y_true"] = y_test_clean
df_test_clean["y_pred_test"] = y_pred_test

# Se crea una columna con booleanos, para determinar si la predicción fue correcta (True = coinciden)
df_test_clean["es_correcta"] = (df_test_clean["y_true"] == df_test_clean["y_pred_test"])

# Se seleccionan solo los errores
errors = df_test_clean[df_test_clean["es_correcta"] == False].copy()

# Se crea una columna con el score, considerando la cantidad de votos y la valoración promedio
errors["score"] = errors["Valoración"] * errors["Nº Votos"]

# Se ordena por este score de mayor a menor y tomar los del top 10
errors_rank10 = errors.sort_values("score", ascending=False).head(10)

# Se muestra el top10 más valorado
errors_rank10

,EPL Id,Título,Autor,Colección,Volumen,Año publicación,Sinopsis,Páginas,Revisión,Idioma,Publicado,Estado,Valoración,Nº Votos,Enlace(s),texto,y_true,y_pred_test,es_correcta,score
1584,4765,Don Quijote de la Mancha (IV CENTENARIO),Miguel de Cervantes Saavedra,NaN,NaN,1605,Poco puede decirse del Quijote que no se haya ...,1511,1.8,Español,A:25-04-2025,Ver.,9.4,256,B0FEB27EF355F81608FD011F90009ADA3F467152,\n\n\n\nEL INGENIOSO CABALLERO DON QUIJOTE DE ...,19,6,False,2406.4
84,2619,El corazón de las tinieblas [T. Araceli e Isabel],Joseph Conrad,NaN,NaN,1899,Tejida en torno a una anécdota mínima —el viaj...,167,2.1,Español,A:10-07-2025,Ver.,8.3,80,94893B9F3B37AE7130A69D15925D3BA4259FF175,\n\n\n\n\nSobre el autor\n\n\n\nJózef Teodor K...,2,6,False,664.0
24611,6777,Tao Te Ching,Lao-Tsé,NaN,NaN,-500,Las enseñanzas del Tao Te Ching o «Libro del S...,92,1.2,Español,A:27-12-2021,Ver.,9.0,64,B1A63FEC8EF77D1758F8279FD5387A216E812B30,\n\n\n\n\r\n Índice de contenido\r\n \n\nC...,17,9,False,576.0
37644,7528,Don Camilo,Giovanni Guareschi,Don Camilo,1.0,1948,En un mundo pequeñito donde se mueven pocos ce...,248,1.4,Español,A:15-04-2020,Disp.,8.6,58,59017CDFE20BD1C1759FB3C2A58A0830FDB6CAF8,"\n\n\n\nAquí,\r\n con tres historias y una re...",25,6,False,498.8
57869,31698,El Hobbit (ilustrado por Jemima Catlin),J. R. R. Tolkien,Legendarium,4.0,1937,Bilbo Bolsón disfruta de una vida sencilla y a...,245,1.0,Español,P:18-07-2016,Disp.,9.3,45,WMVUU247P6QDWM5445URL2RAN2LOVNLD,\n\n\n\n\n\n\n \n\n\n\n\nBilbo Bolsón disfruta...,16,19,False,418.5
29371,19706,El arte de la guerra (v. Thomas Cleary),Sun Tzu,NaN,NaN,-350,La versión de Thomas Cleary de El arte de la g...,64,1.4,Español,A:06-05-2021,Ver.,8.8,46,5C37DA8D071D261AB20F1F298E87BE258E623B5E,\n\n\n\nÍndice de contenido\n\nCubierta\n\n\nE...,3,9,False,404.8
9416,2892,Dersú Uzalá,Vladímir Arséniev,NaN,NaN,1923,Vladímir Arséniev rememora en este libro las a...,294,1.2,Español,A:28-02-2024,Ver.,8.5,44,16F7609537208F85E78FDD991704770AE0A2E504,\n\n\n\n\r\n Índice de contenido\r\n \n\nC...,30,19,False,374.0
10892,4549,La señora Dalloway,Virginia Woolf,NaN,60.0,1925,"En esta novela, la escritora nos transmite una...",171,2.2,Español,A:02-12-2023,Ver.,8.3,43,FE3E17A3594BDBC45786B70C8075F17FEC1F2455,\n\n\n\nPRÓLOGO\n\nVirginia Woolf nace en Lond...,2,4,False,356.9
68239,8416,El jardín secreto,Frances Hodgson Burnett,NaN,NaN,1909,Mary es una niña inglesa nacida en la India. S...,139,1.0,Español,P:21-11-2013,Disp.,8.9,36,626ABF8815DEA4796009AFF54080E88B6F8CC554,\n\n\n\n\n\n\n \n\n\n\n \n\nMary es una niña i...,28,4,False,320.4
252,28881,La sonata a Kreutzer,Lev Nikoláievich Tolstói,NaN,NaN,1889,"Escrita a lo largo de 1889, en una época domin...",124,1.1,Español,A:30-06-2025,Disp.,9.1,28,QVQKX2FA7UU3ANJWRC527Q4DNDM3TNLM,\n\n\n\n\nSobre el autor\n\n\n\nLEV NIKOLÁIEVI...,4,2,False,254.8
